<a href="https://colab.research.google.com/github/secutron/RunTime/blob/master/GPipe0PP_SloMo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### PostP - SloMO / Interp.

from https://github.com/sniklaus/sepconv-slomo

In [1]:
!git clone https://github.com/sniklaus/sepconv-slomo.git VFIASC

Cloning into 'VFIASC'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 223 (delta 6), reused 11 (delta 4), pack-reused 207
Receiving objects: 100% (223/223), 14.50 MiB | 7.38 MiB/s, done.
Resolving deltas: 100% (87/87), done.


In [2]:
%cd /content/VFIASC

/content/VFIASC


In [4]:
from IPython.display import clear_output

!rm -rf network-l1.pytorch
!rm -rf network-lf.pytorch
try:
  !gdown https://drive.google.com/uc?id=1v77wNU8sYh0hBmljeGt5LzY9ehQU0elv
  !gdown https://drive.google.com/uc?id=1na11Ey0TB1KEDps4uEwyTBsDcmpo2kr6
except BaseException:
  !bash download.bash
clear_output()

In [6]:
!pip install cupy
!pip install moviepy
clear_output()

In [8]:
fps_boosting = 'x4' #@param ["x4", "x16"]
%env CUDA_VISIBLE_DEVICES=0
if fps_boosting == 'x4':
  !python run.py --model lf --video '/content/VFIASC/videos/car-turn.mp4' --out ./video_x4.mp4
  video_name = "/content/VFIASC/video_x4.mp4"
elif fps_boosting == 'x16':
  !python run.py --model lf --video '/content/VFIASC/videos/car-turn.mp4' --out ./video_x4.mp4
  !python run.py --model lf --video '/content/VFIASC/video_x4.mp4' --out ./video_x16.mp4
  video_name = "/content/VFIASC/video_x16.mp4"
clear_output()

mfps_of_video = int(cv2.VideoCapture(video_name).get(cv2.CAP_PROP_FPS))
mframes_of_video = int(cv2.VideoCapture(video_name).get(cv2.CAP_PROP_FRAME_COUNT))

In [ ]:
!rm -rf output.mp4
!rm -rf slowed_movie_frames
!rm -rf /content/final.mp4
!rm -rf /content/fps_boosted_video.mp4

fr = int(fps_of_video*int(fps_boosting.split("x")[1]))
!mkdir 'slowed_movie_frames'
vidcap = cv2.VideoCapture(video_name)
success,image = vidcap.read()
count = 0
success = True
while success:
  cv2.imwrite("slowed_movie_frames/frame%09d.jpg" % count, image)
  success,image = vidcap.read()
  count += 1

staffs = []
img = os.listdir("slowed_movie_frames/")
img.sort()
clear_output()
for i in img:
  staffs.append("slowed_movie_frames/"+i)

staff = cv2.imread(staffs[0])  # get size from the 1st frame
writer = cv2.VideoWriter(
    'output.mp4',
    cv2.VideoWriter_fourcc(*'MP4V'),   # codec (*'DIVX', *'MP4V', *'FMP4', *'MJPG', *'XVID', *'MP4S')
    fr,  # fps
    (staff.shape[1], staff.shape[0]),  # width, height
    isColor=len(staff.shape) > 2)
for staff in map(cv2.imread, staffs):
    writer.write(staff)
writer.release()
clear_output()

In [ ]:
!cp -r output.mp4 /content/fps_boosted_video.mp4
!cp -r output.mp4 /content/final.mp4
files.download("/content/fps_boosted_video.mp4")

/content/drive/My Drive/GTask/VOCA/FLAME
